In [2]:
import pandas as pd
from networkx import *
import numpy as np
import networkx as nx
from itertools import combinations
import pickle, matplotlib
from collections import Counter

# Undirected graph

In [3]:
## import data
# data = pd.read_csv(root)
df = pd.read_csv("../data/unambiguous_enron_sentiment.csv", header = 0)
# mean of weights
name = 'Average_sentiment_score'
from_name = 'Name_From'
to_name = 'Name_To'
# averaging without consider direction of from to (used for undirected graph)
df['nodes'] = df[[from_name, to_name]].values.tolist()
df['nodes'] = df['nodes'].apply(lambda x: '--'.join(sorted(map(str, x))))
df = df[['nodes', from_name, to_name, name]].groupby(['nodes'])[name].mean().reset_index()
df[[from_name, to_name]] = df.nodes.apply(
   lambda x: pd.Series(str(x).split("--")))
df[name] = df[name].replace(to_replace = [i for i in set(df[name]) if i>0], value =1)
df[name] = df[name].replace(to_replace = [i for i in set(df[name]) if i<0], value =-1)
df = df.drop(df[df[name] == 0].index)

In [4]:
G_all=nx.from_pandas_edgelist(df,source=from_name, target=to_name, edge_attr=name, create_using=nx.Graph())

In [5]:
c = 0
while(1):
    G_all.remove_nodes_from(list(nx.isolates(G_all)))
    remove = [node for node,degree in G_all.degree() if degree == 1]
    G_all.remove_nodes_from(remove)
    G_all.remove_edges_from(nx.selfloop_edges(G_all, data=True))
    if c == number_of_edges(G_all):
        break
    else:
        c = number_of_edges(G_all)

In [6]:
from itertools import combinations

info = []
negative_edges = dict()
for nodes in combinations(G_all.nodes, 3):
    n_edges = G_all.subgraph(nodes).number_of_edges()
    if n_edges == 3:
        ratio = 1 if np.product([w[name] for n1, n2, w in G_all.subgraph(nodes).edges.data()]) == 1 else 0
        info.append([G_all.subgraph(nodes).edges.data(), 'X', nodes, ratio])
        negative_edges.setdefault([w[name] for n1, n2, w in G_all.subgraph(nodes).edges.data()].count(-1), 0)
        negative_edges[[w[name] for n1, n2, w in G_all.subgraph(nodes).edges.data()].count(-1)]+=1
info = pd.DataFrame(info)
info.to_csv('../info_triads/Enron_sentiment_undirected.csv', index=False)

In [7]:
from collections import Counter
Counter(info[3])

Counter({0: 6990, 1: 14958})

In [8]:
negative_edges

{1: 6687, 0: 12717, 2: 2241, 3: 303}

## Directed Graph

In [9]:
df = pd.read_csv("../data/unambiguous_enron_sentiment.csv", header = 0)
name = 'Average_sentiment_score'
from_name = 'Name_From'
to_name = 'Name_To'
# averaging by from to (used for directed graph)
df = df[[from_name, to_name, name]].groupby([from_name, to_name]).mean().reset_index()

df[name] = df[name].replace(to_replace = [i for i in set(df[name]) if i>0], value =1)
df[name] = df[name].replace(to_replace = [i for i in set(df[name]) if i<0], value =-1)
df = df.drop(df[df[name] == 0].index)
df.shape

(7510, 3)

In [10]:
from itertools import combinations

## Get all triples in triads with respect to their census and edgelists (in edge_atts)
def get_directed_triads(undirected_triad):
    # Get all triplets of edges
    list_tri = []
    for candidate_edges in combinations(undirected_triad.edges.data(), 3):
        # Get edges between unique pair of nodes
        unique_edges = set([tuple(sorted((s,e))) for s,e,w in candidate_edges])
        start_nodes = set([s for s,e,w in candidate_edges])
        # Only consider triad in which the tree edges use a unique pair of nodes
        if len(unique_edges) == 3 and len(start_nodes) !=3:
            yield candidate_edges

In [11]:
G_all=nx.from_pandas_edgelist(df,source=from_name, target=to_name, edge_attr=name, create_using=nx.DiGraph())


In [12]:
c = 0
while(1):
    G_all.remove_nodes_from(list(nx.isolates(G_all)))
    remove = [node for node,degree in G_all.degree() if degree == 1]
    G_all.remove_nodes_from(remove)
    G_all.remove_edges_from(nx.selfloop_edges(G_all, data=True))
    if c == number_of_edges(G_all):
        break
    else:
        c = number_of_edges(G_all)

In [ ]:
# G_all=nx.from_pandas_edgelist(df,source=from_name, target=to_name, edge_attr=name, create_using=nx.DiGraph())
useless_census = ['003','012', '102', '021D', '021C', '021U', '021', '111U', '111D', '201', '030C', '120C', '210']
info = []
balances = []
unbalances = []
for triads in all_triads(G_all):
    if triad_type(triads) not in useless_census:
        all_values = {1:0, 0:0, -1:0}
        for triangle in get_directed_triads(triads):
            all_values[np.product([w[name] for n1, n2, w in triangle])] += 1
        ratio = all_values[1]/(all_values[1]+all_values[-1])
        info.append([triads.edges.data(), triad_type(triads), set([k[0] for k in triads.edges.data()]+[k[1] for k in triads.edges.data()]), ratio])
        
        balance_list = []
        if triad_type(triads) == '300':
            for triangle in get_directed_triads(triads):
                node = []
                for edge in triangle:
                    if edge[0] not in node:
                        node.append(edge[0])
                if len(node) != 3:
                    balance = 1
                    for edge in triangle:
                        balance *= edge[2][name]
                    balance_list.append(balance)
        else:
            for item in get_directed_triads(triads):
                balance = 1
                for edge in item:
                    balance *= edge[2][name]
                balance_list.append(balance)
        if -1 in balance_list:
            unbalances.append(triads)
        else:
            balances.append(triads)

info = pd.DataFrame(info)
info.to_csv('../info_triads/Enron_sentiment_directed.csv', index=False)

In [ ]:
from collections import Counter
Counter(info[3])

In [ ]:
## number of balanced traids
dict_balance = dict()
for i in balances:
    for triangle in get_directed_triads(i):
        if [e[2][name] for e in triangle].count(-1) not in dict_balance.keys():
            dict_balance[[e[2][name] for e in triangle].count(-1)]=0
        dict_balance[[e[2][name] for e in triangle].count(-1)]+=1
print(dict_balance)

In [ ]:
## number of imbalance triads
dict_unbalance = {}
for i in unbalances:
    for triangle in get_directed_triads(i):
        if [e[2][name] for e in triangle].count(-1) not in dict_unbalance.keys():
            dict_unbalance[[e[2][name] for e in triangle].count(-1)]=0
        dict_unbalance[[e[2][name] for e in triangle].count(-1)]+=1
print(dict_unbalance)